# Preprocessing Only Workflow

In this workflow, CFRL takes in an offline trajectory and then preprocesses the offline trajectory 
using `SyntheticPreprocessor`. The final output of the workflow is the preprocessed (debiased) 
offline trajectory. This workflow is appropriate when the user does not want to train policies using 
CFRL. Instead, the user can take the preprocessed trajectory to train a counterfactually fair policy 
using another reinforcement learning library or application that better fits their needs.

We begin by importing the liberaries needed for this demonstration.

In [1]:
# Need this temporarily to import CFRL before it is officially published to PyPI
import sys
sys.path.append("E:/learning/university/MiSIL/CFRL Python Package/CFRL")

In [2]:
import pandas as pd
import numpy as np
#import pytorch as torch
from sklearn.model_selection import train_test_split
from CFRL.reader import read_trajectory_from_dataframe, convert_trajectory_to_dataframe
from CFRL.preprocessor import SequentialPreprocessor
np.random.seed(1) # ensure reproducibility
#torch.manual_seed(1)

## Data Loading

In this demonstration, we use an offline trajectory generated from a `SyntheticEnvironment` using some pre-specified transition rules. Although it is actually synthesized, we treat it as if it is from some unknown environment for pedagogical convenience in this demonstration.

The trajectory contains 500 individuals (i.e. $N=500$) and 10 transitions (i.e. $T=10$). The actions are binary (0 or 1) and were sampled using a random policy that selects 0 or 1 randomly with equal probability. It is stored in a tabular format in a `.csv` file. The sensitive attribute variable is bivariate, stored in columns `z1` and `z2`. The legit values of the sensitive attribute are $[0, 0]$, $[1, 0]$, $[0, 1]$, and $[1, 1]$. The state variable is tri-variate, stored in columns `state1`, `state2`, and `state3`. The actions are stored in the column `action` and rewards in the column `reward`. The tabular data also includes an extra inrrelevant column `timestamp`. 

We can load and view the tabular data.

In [3]:
trajectory = pd.read_csv('../data/sample_data_large_multi.csv')
trajectory

,Unnamed: 0,ID,timestamp,z1,z2,action,reward,state1,state2,state3
0,0,1.0,1.0,0.0,0.0,NaN,NaN,2.124345,-0.111756,-0.028172
1,1,1.0,2.0,0.0,0.0,1.0,3.380339,-0.071876,0.545250,-0.020279
2,2,1.0,3.0,0.0,0.0,0.0,1.849111,-1.084077,-1.696634,-1.179136
3,3,1.0,4.0,0.0,0.0,0.0,-4.421291,-2.317520,-1.787875,-2.148363
4,4,1.0,5.0,0.0,0.0,1.0,-5.142691,-2.936506,-3.603797,-3.590126
...,...,...,...,...,...,...,...,...,...,...
5495,5495,500.0,7.0,0.0,0.0,0.0,-12.563265,-4.024293,-6.587401,-3.859436
5496,5496,500.0,8.0,0.0,0.0,0.0,-14.073520,-5.952644,-5.854450,-4.218220
5497,5497,500.0,9.0,0.0,0.0,0.0,-16.691358,-5.687570,-6.008377,-5.618730
5498,5498,500.0,10.0,0.0,0.0,0.0,-18.394408,-7.551435,-6.816310,-6.740886


We now read the trajectory from the tabular format into Trajectory Arrays.

In [4]:
zs, states, actions, rewards, ids = read_trajectory_from_dataframe(
                                                data=trajectory, 
                                                z_labels=['z1', 'z2'], 
                                                state_labels=['state1', 'state2', 'state3'], 
                                                action_label='action', 
                                                reward_label='reward', 
                                                id_label='ID', 
                                                T=10
                                                )

## Preprocessor Training

Before preprocessing the trajectory, we need to first train a preprocessor. To mitigate overfitting, we use a random subset of 250 individuals in the trajectory to train the preprocessor. The remaining 250 individuals will be actually preprocessed. We now form these two sets.

In [5]:
(
    zs_train, zs_prepro, 
    states_train, states_prepro, 
    actions_train, actions_prepro, 
    rewards_train, rewards_prepro, 
    ids_train, ids_prepro
) = train_test_split(zs, states, actions, rewards, ids, test_size=0.5)

We now use the training set to train a `SequentialPreprocessor`.

In [6]:
sp = SequentialPreprocessor(z_space=[[0, 0], [0, 1], [1, 0], [1, 1]], 
                            num_actions=2, 
                            cross_folds=1, 
                            mode='single', 
                            reg_model='nn')
sp.train_preprocessor(zs=zs_train, xs=states_train, actions=actions_train, rewards=rewards_train)

100%|██████████| 1000/1000 [00:35<00:00, 28.03it/s]


(array([[[ 2.97306092e-01,  6.45651817e-01, -1.68102797e+00, ...,
           2.47447229e+00,  2.52162402e+00,  3.75633999e-01],
         [-1.35034815e+00, -2.25309663e+00, -1.63382889e+00, ...,
           3.26178107e+00,  1.11187966e+00,  1.70306336e+00],
         [-3.42806811e+00, -4.52740321e+00, -3.80843044e+00, ...,
           2.28007655e+00,  1.27067295e+00,  1.28768611e+00],
         ...,
         [-6.80947176e+00, -7.44296927e+00, -6.91436753e+00, ...,
           4.86909207e+00,  3.47462134e+00,  4.05558982e+00],
         [-8.76876788e+00, -7.18474406e+00, -8.88386579e+00, ...,
           3.10601707e+00,  4.60472137e+00,  2.92517047e+00],
         [-9.04944974e+00, -9.08450819e+00, -9.40940367e+00, ...,
           3.63934964e+00,  4.10811876e+00,  3.49294342e+00]],
 
        [[ 3.00060607e-01,  5.07388984e-01,  7.75664075e-01, ...,
           2.47722681e+00,  2.38336118e+00,  2.83232604e+00],
         [-1.27571934e+00, -1.32202527e+00, -2.06041935e+00, ...,
           2.56355566

## Data Preprocessing

We now preprocess the remaining data that are not in the training set.

In [7]:
states_tilde, rewards_tilde = sp.preprocess_multiple_steps(zs=zs_prepro, 
                                                           xs=states_prepro, 
                                                           actions=actions_prepro, 
                                                           rewards=rewards_prepro)

## Data Exporting

Finally, we convert the preprocessed trajectory back into the tabular format so that it is easier to store and manage. For simplicity, we call the new state as `state1`, ..., `state12`, which is the default option provided by `convert_trajectory_to_dataframe` (so we do not need to specify the `state_labels` argument).

In [8]:
preprocessed_trajectory = convert_trajectory_to_dataframe(
                                        zs=zs_prepro, 
                                        states=states_tilde, 
                                        actions=actions_prepro, 
                                        rewards=rewards_tilde, 
                                        ids=ids_prepro, 
                                        z_labels=['z1', 'z2'], 
                                        action_label='action', 
                                        reward_label='reward', 
                                        id_label='ID', 
                                        T_label='time_step'
                                        )
preprocessed_trajectory

,ID,time_step,z1,z2,action,reward,state1,state2,state3,state4,state5,state6,state7,state8,state9,state10,state11,state12
0,305.0,1.0,1.0,0.0,NaN,NaN,-0.317405,0.840769,0.844372,0.459064,2.174768,1.944196,0.454466,1.855285,2.255392,1.859761,2.716742,2.901034
1,305.0,2.0,1.0,0.0,0.0,5.476479,0.758898,1.273863,-3.459000,3.047514,2.411492,-1.208045,2.435313,2.773919,-1.033562,4.036637,4.329050,0.946196
2,305.0,3.0,1.0,0.0,1.0,5.865932,-3.466960,1.151724,-4.185863,0.248516,4.111016,-0.696383,0.170335,2.534116,-1.045698,2.036174,5.102057,0.579744
3,305.0,4.0,1.0,0.0,0.0,2.696097,-1.720059,-2.800422,-3.303409,1.650113,1.074845,0.993773,0.223361,-0.293243,0.088610,3.147963,2.171222,2.949789
4,305.0,5.0,1.0,0.0,1.0,2.246844,-1.905602,-2.679749,-3.108612,1.817850,1.633375,0.851067,1.239327,0.528893,0.176984,4.931711,4.689713,4.121794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2745,473.0,7.0,1.0,0.0,1.0,3.443737,-4.566324,-4.944158,-5.746439,0.588494,1.946760,-0.218327,1.912245,0.770409,0.296687,6.638594,5.992071,5.271190
2746,473.0,8.0,1.0,0.0,0.0,0.691425,-6.988944,-7.163942,-7.099005,-1.100196,-1.182431,-0.859120,-1.036336,-0.033916,-1.039807,4.731120,4.683079,4.290400
2747,473.0,9.0,1.0,0.0,1.0,-0.176195,-8.535959,-7.307201,-6.528395,-1.844173,-0.478055,-0.634874,-2.017924,-1.247170,-0.657567,4.336152,5.748313,5.775954
2748,473.0,10.0,1.0,0.0,1.0,-1.898042,-8.535721,-8.361477,-10.089605,-1.496266,-0.836802,-3.871994,-1.900819,-1.737821,-3.682403,4.799341,5.822716,3.642264


## Alternative: Preprocessing All Individuals

Sometimes, the number of individuals in the trajectory is small. In this case, if we only preprocess a subset of individuals, the resulting preprocessed trajectory might be too small to be useful. In this case, we can directly preprocess all individuals using the `train_preprocessor()` function when we have a relatively large number of `cross_folds`.

When `cross_folds=K` where `K` is greater than 1, `train_preprocessor()` will internally divide the training data into `K` folds. For each $i=1,\dots,k$, it trains a model based on all the folds other than the $i$-th one, which is then used to preprocess data in the $i$-th fold. This results in `K` folds of preprocessed data, each of which is processed using a model that is trained on the other folds. These `K` folds of data are then combined and returned by `train_preprocessor()`. This method allows us to preprocess all individuals in the trajectory while reducing overfitting.

To use this functionality, we first initialize a `SequentialPreprocessor` with `cross_folds` greater than 1. We use `cross_folds=5` here.

In [9]:
sp_cf5 = SequentialPreprocessor(z_space=[[0, 0], [0, 1], [1, 0], [1, 1]], 
                                num_actions=2, 
                                cross_folds=5, 
                                mode='single', 
                                reg_model='nn')

We now simultaneously train the preprocessor and preprocess all individuals in the trajectory using the precedure described above.

In [10]:
states_tilde_cf5, rewards_tilde_cf5 = sp_cf5.train_preprocessor(zs=zs, 
                                                                xs=states, 
                                                                actions=actions, 
                                                                rewards=rewards)

100%|██████████| 1000/1000 [00:57<00:00, 17.54it/s]


We can now convert the preprocessed trajectory into the tabular format.

In [11]:
preprocessed_trajectory_cf5 = convert_trajectory_to_dataframe(
                                            zs=zs_prepro, 
                                            states=states_tilde, 
                                            actions=actions_prepro, 
                                            rewards=rewards_tilde, 
                                            ids=ids_prepro, 
                                            z_labels=['z1', 'z2'], 
                                            action_label='action', 
                                            reward_label='reward', 
                                            id_label='ID', 
                                            T_label='time_step'
                                            )
preprocessed_trajectory_cf5

,ID,time_step,z1,z2,action,reward,state1,state2,state3,state4,state5,state6,state7,state8,state9,state10,state11,state12
0,305.0,1.0,1.0,0.0,NaN,NaN,-0.317405,0.840769,0.844372,0.459064,2.174768,1.944196,0.454466,1.855285,2.255392,1.859761,2.716742,2.901034
1,305.0,2.0,1.0,0.0,0.0,5.476479,0.758898,1.273863,-3.459000,3.047514,2.411492,-1.208045,2.435313,2.773919,-1.033562,4.036637,4.329050,0.946196
2,305.0,3.0,1.0,0.0,1.0,5.865932,-3.466960,1.151724,-4.185863,0.248516,4.111016,-0.696383,0.170335,2.534116,-1.045698,2.036174,5.102057,0.579744
3,305.0,4.0,1.0,0.0,0.0,2.696097,-1.720059,-2.800422,-3.303409,1.650113,1.074845,0.993773,0.223361,-0.293243,0.088610,3.147963,2.171222,2.949789
4,305.0,5.0,1.0,0.0,1.0,2.246844,-1.905602,-2.679749,-3.108612,1.817850,1.633375,0.851067,1.239327,0.528893,0.176984,4.931711,4.689713,4.121794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2745,473.0,7.0,1.0,0.0,1.0,3.443737,-4.566324,-4.944158,-5.746439,0.588494,1.946760,-0.218327,1.912245,0.770409,0.296687,6.638594,5.992071,5.271190
2746,473.0,8.0,1.0,0.0,0.0,0.691425,-6.988944,-7.163942,-7.099005,-1.100196,-1.182431,-0.859120,-1.036336,-0.033916,-1.039807,4.731120,4.683079,4.290400
2747,473.0,9.0,1.0,0.0,1.0,-0.176195,-8.535959,-7.307201,-6.528395,-1.844173,-0.478055,-0.634874,-2.017924,-1.247170,-0.657567,4.336152,5.748313,5.775954
2748,473.0,10.0,1.0,0.0,1.0,-1.898042,-8.535721,-8.361477,-10.089605,-1.496266,-0.836802,-3.871994,-1.900819,-1.737821,-3.682403,4.799341,5.822716,3.642264
